In [10]:
import copy
import glob
import regex as re
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%load_ext rpy2.ipython
%run ../airquality/data/gen_daily_targets.py
%run ../airquality/data/prepare_data.py

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
test = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')

In [5]:
observations = pd.read_csv('../data/observations.csv', index_col=0)
models = pd.read_csv('../data/models.csv', index_col=0)
obs_and_mods = pd.merge(models, observations, how='left',
                    on=['station', 'day', 'time', 'datetime' ,'year'],
                    suffixes=('_mod', '')).sort_values('datetime', ascending=True)
obs_and_mods.to_csv('../data/obs_and_mod.csv')
obs_and_mods_cols = ['pred_0_days', 'pred_1_days', 'Concentration', 'target', 'day']
obs_and_mods.head()

,pred_0_days,pred_1_days,day,lon,lat,year,station,datetime,time,Concentration,target
0,38.79,34.11,2013-01-01,2.15,41.39,2013,ES1438A,2013-01-01 00:00:00,00:00:00,nan,nan
1,28.53,27.48,2013-01-01,2.13,41.38,2013,ES1396A,2013-01-01 00:00:00,00:00:00,nan,nan
2,35.85,42.57,2013-01-01,2.20,41.40,2013,ES0691A,2013-01-01 00:00:00,00:00:00,nan,nan
3,31.81,31.59,2013-01-01,2.15,41.40,2013,ES1480A,2013-01-01 00:00:00,00:00:00,nan,nan
4,31.81,31.59,2013-01-01,2.15,41.43,2013,ES1856A,2013-01-01 00:00:00,00:00:00,nan,nan


In [6]:
agg_types = ['mean', 'max', 'std']
tall_series = obs_and_mods.groupby(['day', 'station']).agg({
    'Concentration': agg_types
})['Concentration'].reset_index()
aggs = [tall_series.pivot(index='day', columns='station', values=agg) for agg in agg_types]
aggs = [df.rename(columns={c: c + '_' + agg for c in df.columns}) for df, agg in zip(aggs, agg_types)]
wide_series = pd.concat(aggs, axis=1)
wide_series.head()

station,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2013-01-01,42.39,38.61,51.39,43.87,37.13,14.78,23.35,72.00,98.00,112.00,...,69.00,39.00,68.00,18.13,26.67,25.73,24.63,20.29,10.99,21.15
2013-01-02,52.50,47.58,63.71,54.50,48.75,32.08,43.29,87.00,86.00,119.00,...,76.00,77.00,85.00,16.61,24.72,26.48,19.18,16.91,25.43,26.53
2013-01-03,64.62,54.62,74.04,64.17,49.25,38.88,49.04,119.00,85.00,117.00,...,78.00,94.00,110.00,22.69,19.05,22.87,21.70,14.87,22.17,31.43
2013-01-04,54.12,44.30,44.50,89.13,61.92,22.18,39.04,80.00,113.00,123.00,...,108.00,60.00,99.00,16.58,28.35,32.46,45.41,25.83,16.98,28.20
2013-01-05,36.88,39.17,nan,89.00,64.00,12.25,33.54,60.00,114.00,nan,...,114.00,16.00,85.00,11.00,22.21,nan,21.63,17.04,2.56,17.75


In [16]:
%%time
lags = 100
preds_by_station = []
edit_wide_series = copy.deepcopy(wide_series)
for d in test['date'].unique():
    data = edit_wide_series[list(pd.to_datetime(edit_wide_series.reset_index().day) <= d)]
    for s in test['station'].unique():
        to_lag = data[[c for c in data.columns if not c in ['{}_{}'.format(s, agg) for agg in agg_types]]]
        X = create_lagged_features(to_lag, lags)
        X_train = np.nan_to_num(X.iloc[:-1].values)
        for agg in agg_types:
            print('Predicting {} for station {} day {}'.format(agg, s, d))
            y_train = np.nan_to_num(data['{}_{}'.format(s, agg)][:-1])
            X_test = np.nan_to_num(X.iloc[-1].values.reshape(1, X.shape[1]))
            model = xgb.XGBRegressor(n_estimators=100, n_jobs=4)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            edit_wide_series.loc[d, '{}_{}'.format(s, agg)] = y_pred
            print(y_pred)

Predicting mean for station ES0691A day 2015-01-03
[ 43.07396698]
Predicting max for station ES0691A day 2015-01-03
[ 93.66294098]
Predicting std for station ES0691A day 2015-01-03
[ 20.30436707]
Predicting mean for station ES1396A day 2015-01-03
[ 41.48853302]
Predicting max for station ES1396A day 2015-01-03
[ 73.35262299]
Predicting std for station ES1396A day 2015-01-03
[ 17.78541183]
Predicting mean for station ES1438A day 2015-01-03
[ 61.82097244]
Predicting max for station ES1438A day 2015-01-03
[ 106.2568512]
Predicting std for station ES1438A day 2015-01-03
[ 20.35978889]
Predicting mean for station ES1480A day 2015-01-03
[ 61.06529236]
Predicting max for station ES1480A day 2015-01-03
[ 114.90940094]
Predicting std for station ES1480A day 2015-01-03
[ 24.06773758]
Predicting mean for station ES1679A day 2015-01-03
[ 44.8682785]
Predicting max for station ES1679A day 2015-01-03
[ 87.08092499]
Predicting std for station ES1679A day 2015-01-03
[ 17.87788773]
Predicting mean for 

[ 18.69677353]
Predicting mean for station ES0691A day 2015-01-21
[ 34.96103287]
Predicting max for station ES0691A day 2015-01-21
[ 64.34474182]
Predicting std for station ES0691A day 2015-01-21
[ 15.65393257]
Predicting mean for station ES1396A day 2015-01-21
[ 28.22834778]
Predicting max for station ES1396A day 2015-01-21
[ 60.92477417]
Predicting std for station ES1396A day 2015-01-21
[ 12.67709064]
Predicting mean for station ES1438A day 2015-01-21
[ 50.85324097]
Predicting max for station ES1438A day 2015-01-21
[ 89.8829422]
Predicting std for station ES1438A day 2015-01-21
[ 19.06084824]
Predicting mean for station ES1480A day 2015-01-21
[ 52.3229599]
Predicting max for station ES1480A day 2015-01-21
[ 84.32099152]
Predicting std for station ES1480A day 2015-01-21
[ 16.84487343]
Predicting mean for station ES1679A day 2015-01-21
[ 34.30517578]
Predicting max for station ES1679A day 2015-01-21
[ 64.48420715]
Predicting std for station ES1679A day 2015-01-21
[ 15.58916283]
Predict

[ 17.26331711]
Predicting mean for station ES0691A day 2015-02-08
[ 29.65516663]
Predicting max for station ES0691A day 2015-02-08
[ 63.67966843]
Predicting std for station ES0691A day 2015-02-08
[ 14.91178799]
Predicting mean for station ES1396A day 2015-02-08
[ 22.69000244]
Predicting max for station ES1396A day 2015-02-08
[ 65.0928421]
Predicting std for station ES1396A day 2015-02-08
[ 13.52116966]
Predicting mean for station ES1438A day 2015-02-08
[ 50.83269119]
Predicting max for station ES1438A day 2015-02-08
[ 98.09896851]
Predicting std for station ES1438A day 2015-02-08
[ 20.11649323]
Predicting mean for station ES1480A day 2015-02-08
[ 45.05382156]
Predicting max for station ES1480A day 2015-02-08
[ 79.06079102]
Predicting std for station ES1480A day 2015-02-08
[ 17.30606461]
Predicting mean for station ES1679A day 2015-02-08
[ 29.56731987]
Predicting max for station ES1679A day 2015-02-08
[ 62.49032211]
Predicting std for station ES1679A day 2015-02-08
[ 14.70467472]
Predic

[ 22.78032112]
Predicting mean for station ES0691A day 2015-02-26
[ 44.85150528]
Predicting max for station ES0691A day 2015-02-26
[ 74.30310059]
Predicting std for station ES0691A day 2015-02-26
[ 16.62187195]
Predicting mean for station ES1396A day 2015-02-26
[ 36.06340408]
Predicting max for station ES1396A day 2015-02-26
[ 80.44622803]
Predicting std for station ES1396A day 2015-02-26
[ 16.9215641]
Predicting mean for station ES1438A day 2015-02-26
[ 58.33491516]
Predicting max for station ES1438A day 2015-02-26
[ 97.94306183]
Predicting std for station ES1438A day 2015-02-26
[ 18.86869621]
Predicting mean for station ES1480A day 2015-02-26
[ 58.77519989]
Predicting max for station ES1480A day 2015-02-26
[ 118.76908112]
Predicting std for station ES1480A day 2015-02-26
[ 24.04342842]
Predicting mean for station ES1679A day 2015-02-26
[ 43.25999832]
Predicting max for station ES1679A day 2015-02-26
[ 76.47362518]
Predicting std for station ES1679A day 2015-02-26
[ 17.24762917]
Predi

[ 19.92846107]
Predicting mean for station ES0691A day 2015-03-16
[ 51.41650391]
Predicting max for station ES0691A day 2015-03-16
[ 86.77796173]
Predicting std for station ES0691A day 2015-03-16
[ 17.93181229]
Predicting mean for station ES1396A day 2015-03-16
[ 40.39534378]
Predicting max for station ES1396A day 2015-03-16
[ 68.71697235]
Predicting std for station ES1396A day 2015-03-16
[ 16.15117836]
Predicting mean for station ES1438A day 2015-03-16
[ 59.32090759]
Predicting max for station ES1438A day 2015-03-16
[ 116.79650879]
Predicting std for station ES1438A day 2015-03-16
[ 20.02294159]
Predicting mean for station ES1480A day 2015-03-16
[ 50.17444229]
Predicting max for station ES1480A day 2015-03-16
[ 94.1977005]
Predicting std for station ES1480A day 2015-03-16
[ 20.11052322]
Predicting mean for station ES1679A day 2015-03-16
[ 42.42429733]
Predicting max for station ES1679A day 2015-03-16
[ 79.40562439]
Predicting std for station ES1679A day 2015-03-16
[ 18.84492874]
Predi

...

[ 19.68225861]
Predicting mean for station ES0691A day 2015-10-18
[ 39.30297089]
Predicting max for station ES0691A day 2015-10-18
[ 73.41216278]
Predicting std for station ES0691A day 2015-10-18
[ 16.46599579]
Predicting mean for station ES1396A day 2015-10-18
[ 30.86940956]
Predicting max for station ES1396A day 2015-10-18
[ 63.05483246]
Predicting std for station ES1396A day 2015-10-18
[ 14.92583656]
Predicting mean for station ES1438A day 2015-10-18
[ 53.25634003]
Predicting max for station ES1438A day 2015-10-18
[ 87.18281555]
Predicting std for station ES1438A day 2015-10-18
[ 16.10367584]
Predicting mean for station ES1480A day 2015-10-18
[ 51.2550621]
Predicting max for station ES1480A day 2015-10-18
[ 90.21167755]
Predicting std for station ES1480A day 2015-10-18
[ 19.55840683]
Predicting mean for station ES1679A day 2015-10-18
[ 37.01793671]
Predicting max for station ES1679A day 2015-10-18
[ 74.31533051]
Predicting std for station ES1679A day 2015-10-18
[ 17.77015877]
Predic

[ 12.43632507]
Predicting mean for station ES0691A day 2015-11-05
[ 43.8226738]
Predicting max for station ES0691A day 2015-11-05
[ 79.63323975]
Predicting std for station ES0691A day 2015-11-05
[ 19.04608727]
Predicting mean for station ES1396A day 2015-11-05
[ 36.72262955]
Predicting max for station ES1396A day 2015-11-05
[ 86.85466766]
Predicting std for station ES1396A day 2015-11-05
[ 20.92216873]
Predicting mean for station ES1438A day 2015-11-05
[ 59.49800491]
Predicting max for station ES1438A day 2015-11-05
[ 121.3850708]
Predicting std for station ES1438A day 2015-11-05
[ 24.38304329]
Predicting mean for station ES1480A day 2015-11-05
[ 59.03992081]
Predicting max for station ES1480A day 2015-11-05
[ 109.55288696]
Predicting std for station ES1480A day 2015-11-05
[ 23.58354759]
Predicting mean for station ES1679A day 2015-11-05
[ 46.07226944]
Predicting max for station ES1679A day 2015-11-05
[ 85.09210968]
Predicting std for station ES1679A day 2015-11-05
[ 19.34829712]
Predi

[ 20.85770416]
Predicting mean for station ES0691A day 2015-11-23
[ 54.90533447]
Predicting max for station ES0691A day 2015-11-23
[ 88.57671356]
Predicting std for station ES0691A day 2015-11-23
[ 15.30515575]
Predicting mean for station ES1396A day 2015-11-23
[ 45.45676422]
Predicting max for station ES1396A day 2015-11-23
[ 79.73672485]
Predicting std for station ES1396A day 2015-11-23
[ 19.18790436]
Predicting mean for station ES1438A day 2015-11-23
[ 61.99214172]
Predicting max for station ES1438A day 2015-11-23
[ 108.84117126]
Predicting std for station ES1438A day 2015-11-23
[ 23.72223473]
Predicting mean for station ES1480A day 2015-11-23
[ 68.73806]
Predicting max for station ES1480A day 2015-11-23
[ 116.20943451]
Predicting std for station ES1480A day 2015-11-23
[ 26.09166336]
Predicting mean for station ES1679A day 2015-11-23
[ 49.88312912]
Predicting max for station ES1679A day 2015-11-23
[ 84.53817749]
Predicting std for station ES1679A day 2015-11-23
[ 16.71679306]
Predic

[ 19.30486679]
Predicting mean for station ES0691A day 2015-12-11
[ 56.52889252]
Predicting max for station ES0691A day 2015-12-11
[ 90.67118835]
Predicting std for station ES0691A day 2015-12-11
[ 17.72316742]
Predicting mean for station ES1396A day 2015-12-11
[ 43.6562233]
Predicting max for station ES1396A day 2015-12-11
[ 75.98384857]
Predicting std for station ES1396A day 2015-12-11
[ 16.88642502]
Predicting mean for station ES1438A day 2015-12-11
[ 65.55641174]
Predicting max for station ES1438A day 2015-12-11
[ 109.18273926]
Predicting std for station ES1438A day 2015-12-11
[ 20.51414299]
Predicting mean for station ES1480A day 2015-12-11
[ 75.84629822]
Predicting max for station ES1480A day 2015-12-11
[ 109.76506042]
Predicting std for station ES1480A day 2015-12-11
[ 19.08623886]
Predicting mean for station ES1679A day 2015-12-11
[ 47.6388855]
Predicting max for station ES1679A day 2015-12-11
[ 76.99765778]
Predicting std for station ES1679A day 2015-12-11
[ 18.45972252]
Predi

[ 24.225317]
Predicting mean for station ES0691A day 2015-12-29
[ 62.85308838]
Predicting max for station ES0691A day 2015-12-29
[ 96.74650574]
Predicting std for station ES0691A day 2015-12-29
[ 18.01958847]
Predicting mean for station ES1396A day 2015-12-29
[ 56.70668411]
Predicting max for station ES1396A day 2015-12-29
[ 101.05841064]
Predicting std for station ES1396A day 2015-12-29
[ 20.90931511]
Predicting mean for station ES1438A day 2015-12-29
[ 67.72902679]
Predicting max for station ES1438A day 2015-12-29
[ 118.22734833]
Predicting std for station ES1438A day 2015-12-29
[ 19.4281826]
Predicting mean for station ES1480A day 2015-12-29
[ 77.30073547]
Predicting max for station ES1480A day 2015-12-29
[ 123.88948059]
Predicting std for station ES1480A day 2015-12-29
[ 23.93195534]
Predicting mean for station ES1679A day 2015-12-29
[ 59.61474609]
Predicting max for station ES1679A day 2015-12-29
[ 101.44898224]
Predicting std for station ES1679A day 2015-12-29
[ 21.98204994]
Pred

In [257]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

kf = KFold(n_splits=3)
metric_by_station = []
for s in observations.station.unique():
    data = pd.merge(obs_and_mods[obs_and_mods.station == s].set_index('datetime')[obs_and_mods_cols],
                    lagged_features,
                    right_index=True, left_index=True, how='left').dropna()
    X = data[[c for c in data.columns if not c in ['datetime', 'day', 'time', 'station', 'year', 'target']]].values
    y = data[['target']].values.ravel()
    metric = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = xgb.XGBClassifier(n_estimators=100)
        model.fit(X_train, y_train)
        y_pred = [int(e) for e in model.predict(X_test)]
        score = log_loss(y_test, y_pred, labels=(0, 1))
        metric.append(score) 
        
    metric_by_station.append((s, np.mean(metric)))
    
metric_by_station

[('ES1992A', 1.1188109386021232),
 ('ES1480A', 3.0023406464819105),
 ('ES1856A', 0.43173470493638444),
 ('ES1396A', 0.65453614530276116),
 ('ES0691A', 0.28861262014947991),
 ('ES1438A', 2.9568844298019088),
 ('ES1679A', 0.21519486850411737)]